# Integrate LlamaIndex with Astra DB Serverless

For more information, visit the DataStax [Astra DB docs page](https://docs.datastax.com/en/astra/astra-db-vector/integrations/llamaindex.html).

In [ ]:
! pip install --quiet "llama-index==0.10.15" \
  "llama-index-vector-stores-astra-db>=0.1.3" \
  "python-dotenv==1.0.1"

## Secrets

Example values:
- API Endpoint: `"https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com"`
- Token: `"AstraCS:6gBhNmsk135..."` (it must have a role of at least "Database Administrator")
- OpenAI API key: `sk-4fQ3F...`

In [ ]:
import os
from getpass import getpass
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")
os.environ["ASTRA_DB_API_ENDPOINT"] = input("ASTRA_DB_API_ENDPOINT = ")
os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY = ")

## Dependencies

In [ ]:
import os

from llama_index.vector_stores.astra_db import AstraDBVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.llama_dataset import download_llama_dataset

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = os.environ.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_API_ENDPOINT = os.environ.get("ASTRA_DB_API_ENDPOINT")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## Download and inspect LlamaHub dataset

In [ ]:
rag_dataset, documents = download_llama_dataset(
  "PaulGrahamEssayDataset", "./data"
)

print(f"Number of loaded documents: {len(documents)}")
print(f"First document, id: {documents[0].doc_id}")
print(f"First document, hash: {documents[0].hash}")
print(
    "First document, text"
    f" ({len(documents[0].text)} characters):\n"
    f"{'=' * 20}\n"
    f"{documents[0].text[:360]} ..."
)

## (Optional) See chunking in action

_This can be skipped, it's here just for a quick peek at what LlamaIndex does internally upon document ingestion._

In [ ]:
# This step is optional because splitting happens automatically during ingestion
from llama_index.core.node_parser import SentenceSplitter
default_splitter = SentenceSplitter()
split_nodes = default_splitter(documents)
print(f"Number of split nodes: {len(split_nodes)}")
print(f"Third split node, document reference ID: {split_nodes[2].ref_doc_id}")
print(f"Third split node, node ID: {split_nodes[2].node_id}")
print(f"Third split node, hash: {split_nodes[2].hash}")
print(
    "Third split node, text"
    f" ({len(split_nodes[2].text)} characters):\n"
    f"{'=' * 20}\n"
    f"{split_nodes[2].text[:360]} ..."
)

## Create an Astra DB vector store (with its collection on DB)

In [ ]:
astra_db_store = AstraDBVectorStore(
    token=ASTRA_DB_APPLICATION_TOKEN,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    collection_name="llama_index_rag_test",
    embedding_dimension=1536,
)

## Load the documents in the vector store

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

index = VectorStoreIndex.from_documents(
    documents=documents, storage_context=storage_context
)

## Run a similarity search (to verify the integration)

In [ ]:
query_engine = index.as_query_engine()
query_string_1 = "Why did the author choose to work on AI?"
response = query_engine.query(query_string_1)

print("\n\n" + query_string_1)
print(response.response)

## Further usage patterns

### MMR (maximal marginal relevance) similarity search

In [ ]:
retriever = index.as_retriever(
    vector_store_query_mode="mmr",
    similarity_top_k=3,
    vector_store_kwargs={"mmr_prefetch_factor": 4},
)

query_string_2 = "Why did the author choose to work on AI?"
nodes_with_scores = retriever.retrieve(query_string_2)

print("\n\n" + query_string_2 + " (question asked with MMR)")
print(f"Found {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f"    [{idx}] score = {node_with_score.score}")
    print(f"        id    = {node_with_score.node.node_id}")
    print(f"        text  = {node_with_score.node.text[:90]} ...")